Lab 3

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats


In [ ]:
# grab the data from online, Excel is fine
url = 'https://www.qogdata.pol.gu.se/data/qog_bas_cs_jan24.xlsx'
df = pd.read_excel(url)

In [ ]:
df.head()

,ccode,cname,ccode_qog,cname_qog,ccodealp,ccodecow,version,ajr_settmort,atop_ally,atop_number,...,wvs_imprel,wvs_pmi12,wvs_psarmy,wvs_psdem,wvs_psexp,wvs_pssl,wvs_relacc,wvs_satfin,wvs_subh,wvs_trust
0,4,Afghanistan,4,Afghanistan,AFG,700.0,QoGBasCSjan24,4.540098,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,Albania,8,Albania,ALB,339.0,QoGBasCSjan24,NaN,1.0,8.0,...,2.869328,NaN,1.596485,3.849031,3.475513,1.744196,NaN,NaN,3.488758,0.027857
2,12,Algeria,12,Algeria,DZA,615.0,QoGBasCSjan24,4.359270,1.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20,Andorra,20,Andorra,AND,232.0,QoGBasCSjan24,NaN,1.0,2.0,...,2.034930,2.710393,1.336049,3.681363,2.635721,1.830491,1.751004,6.561316,4.089642,0.255744
4,24,Angola,24,Angola,AGO,540.0,QoGBasCSjan24,5.634789,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**1--  Run a simple bivariate regression, and interpret your results. (Did the results fit your expectations? Why? Why not?)**


Dependent Variable: GDP per capita (2015 US dollars) (wdi_gdpcapcon2015).  You can see the distribution of it here.

In [ ]:
df[['wdi_gdpcapcon2015']].describe()

,wdi_gdpcapcon2015
count,188.000000
mean,13726.184306
std,21112.612680
min,263.360992
25%,1939.369629
50%,5395.598633
75%,15333.187744
max,166910.625000


Independent Variable: Corruption Perceptions Index (ti_cpi), Scale of 0-100 where 0 equals the highest level of perceived corruption and 100 equals the lowest level of perceived corruption.  This is the description of the corruption index.  



In [ ]:
df[['ti_cpi']].describe()

,ti_cpi
count,179.000000
mean,43.262570
std,18.771341
min,12.000000
25%,29.000000
50%,40.000000
75%,56.000000
max,88.000000


I expect that there is a relationship where economies perceived as less corrupt (by their populaces) tend to have higher GDP per capita.  (Notice the subset=df['vdem_corr'].notna() element below, which restricts the analysis to countries that are not missing on this other variable, vdem_corr, which I am going to add in the next analysis, and needs to be not missing.)

In [ ]:
corruption_gdp = smf.ols(formula = 'wdi_gdpcapcon2015~ti_cpi', data = df, subset=df['vdem_corr'].notna()).fit()
print (corruption_gdp.summary())

                            OLS Regression Results                            
Dep. Variable:      wdi_gdpcapcon2015   R-squared:                       0.632
Model:                            OLS   Adj. R-squared:                  0.630
Method:                 Least Squares   F-statistic:                     285.1
Date:                Sat, 08 Jun 2024   Prob (F-statistic):           7.10e-38
Time:                        16:11:23   Log-Likelihood:                -1804.9
No. Observations:                 168   AIC:                             3614.
Df Residuals:                     166   BIC:                             3620.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.126e+04   2203.355     -9.649      0.0

Coefficient for Corruption Perceptions Index is 790, which indicates that for each additional point in the Corruption Perceptions Index suggests lower perceived corruption, such that the GDP per capita increases by approximately 790 USD. The model is statistically significant with p < 0.001. The R-squared of 0.63 indicats a strong association.

Indeed, the regression suggests a relationship where economies perceived as less corrupt tend to have higher GDP per capita.

**2--  Add an additional variable that might mediate or partly "explain" the initial association from that simple regression above -- and explain your results. Did it work out? Yes? No?**


Political Corruption Index (vdem_corr) is added.  It ranges from 0 (less corrupt) to 1 (more corrupt). This will allow us to see if an *actual* corruption index (designed by political science experts) can explain the association between GDP and *perceived* corruption (by the populaces).

But it is confusing that this measure of corruption, the higher the score, the more corrupt, but the previous measure of perceived corruption, the higher the score the less corrupt -- so I am going to recode this new measure to go from high corruption (0) to low corrpution (100).  See that distribution below.

In [ ]:
df['new_vdem_corr'] = (1- df['vdem_corr'])*100

In [ ]:
df[['new_vdem_corr']].describe()

,new_vdem_corr
count,173.000000
mean,51.012139
std,30.047921
min,4.799998
25%,23.600000
50%,46.300000
75%,77.800000
max,99.800000


How do my two independent variables (two measure of corrpution) correlate?  They correlate highly (+0.91).

In [ ]:
df_filtered = df[['ti_cpi', 'new_vdem_corr']].dropna()
stats.pearsonr(df_filtered['ti_cpi'], df_filtered['new_vdem_corr'])


PearsonRResult(statistic=0.9073063660295007, pvalue=3.0565534424885073e-66)

So this regression will involve the political Corruption Index (vdem_corr) is added, Ranges from 0 (very corrupt) to 100 (very un-corrupt). This will allow us to see if an *actual* corruption index (designed by political science experts) can explain the association between GDP and *perceived* corruption.

In [ ]:
corruption_actual_gdp = smf.ols(formula = 'wdi_gdpcapcon2015 ~ ti_cpi + new_vdem_corr', data = df).fit()
print (corruption_actual_gdp.summary())

                            OLS Regression Results                            
Dep. Variable:      wdi_gdpcapcon2015   R-squared:                       0.648
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     151.9
Date:                Sat, 08 Jun 2024   Prob (F-statistic):           3.82e-38
Time:                        20:05:43   Log-Likelihood:                -1801.1
No. Observations:                 168   AIC:                             3608.
Df Residuals:                     165   BIC:                             3618.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -2.336e+04   2292.757    -10.188

Corruption Perceptions Index has a coefficient of 1059, indicating that a 1 point increase in better perceived corruption (where it goes up to 100) is associated with an increase of $1059 in GDP per capita, net of actual corruption.

Political Corruption Index has a coefficient of -185.03, suggesting that as political corruption improves or as new_vdem_corr increases by 1 unit (out of a scale that goes to 100), GDP per capita tends to *decrease* by approximately $185, net of perceived corruption.  Hmmm?

The adjusted R-squared is 0.644, indicating that about 64.4% of the variance in GDP per capita is explained by the two variables combined.
Both corruption variables are statistically significance, with p-values < 0.01.

This result seems counterintuitive and suggests that while perceived corruption and actual corruption are highly correlated, when they differ, their effects also differ, with lower perceived corruption being beneficial for GDP, but lower actual political corruption *lowers* GDP.

**3-- More on extreme combinations.  Find the top 5 entities that are ranked at the top on one variable and ranked at the bottom on another variable.  Interpret your results.**

I am going to look to countries that are very high in their human capital formation, but do not appear to have very much political participation among their citizenry.

In [ ]:
df['human_capital_rank'] = df['pwt_hci'].rank(ascending=False)
df['political_participation_rank'] = df['bti_pp'].rank(ascending=True)

extreme_comb = df[['cname', 'human_capital_rank', 'political_participation_rank']].copy()
extreme_comb['rank_difference'] = extreme_comb['political_participation_rank'] - extreme_comb['human_capital_rank']
extreme_sorted = extreme_comb.sort_values(by='rank_difference', ascending=False).head(5)

extreme_sorted

,cname,human_capital_rank,political_participation_rank,rank_difference
156,Slovakia,3.0,131.5,128.5
46,Czechia,10.0,131.5,121.5
56,Estonia,12.0,131.5,119.5
90,Korea (the Republic of),5.0,118.0,113.0
43,Croatia,14.0,118.0,104.0


These results highlight a significant difference in these countries between the level of human capital and political participation. This could suggest a potential inefficiency or barrier in translating high human capital into political engagement.  It is interesting that so many of these countries were formerly communist countries.